# Practical Session 2
### Kernel Methods for Machine Learning

Written by Yunlong Jiao / Romain Menegaux, 19 May 2020

In [1]:
# setup
import numpy as np

In [2]:
import sys
print(sys.version)

2.7.14 |Anaconda, Inc.| (default, Oct  5 2017, 02:28:52) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [3]:
import sklearn
from sklearn import linear_model as lm
sklearn.__version__

'0.19.0'

***
## Tasks

1. Implement (naive) solvers to Ridge Regression, Weighted Ridge Regression and Logistic Ridge Regression (using Iteratively Reweighted Least Squares). See notes for the mathematical derivation.
2. Simulate some toy data to check if our solvers give correct solutions.

In [4]:
# Toy data
np.random.seed(42) # for reproducibility
n = 100 # number of samples
p = 10 # number of features
X = np.random.normal(0, 1, (n, p))
X = sklearn.preprocessing.scale(X) # scale to 0 mean and 1 standard deviation
beta_star = np.random.normal(0, 1, p)
y = X.dot(beta_star) + 0.2 * np.random.normal(0, 1, n) # Xbeta + noise

def compare(beta1, beta2):
    print('''
Our solver:
{}
Scikit-learn:
{}

Difference between the two:
{}
        '''.format(beta1, beta2, np.sum((beta1-beta2)**2))
    )

***
### Ridge Regression (RR)

Given $X \in \mathbb{R}^{n \times p}$ and $y \in \mathbb{R}^n$, solve
$$
\min_{\beta \in \mathbb{R}^p} \frac{1}{n} \|y - X \beta\|^2 + \lambda \|\beta\|^2 \,.
$$

The gradient (Jacobian) of the objective is:
$$\nabla f(\beta) = \frac{2}{n}X^\top(X\beta - y) + 2\lambda \beta$$

The optimal solution verifies:
<font color='green'>
$$\nabla f(\beta) = 0 \iff (X^\top X + n\lambda I)\beta = X^\top y$$
</font>

$\|y - X \beta\|^2 = (X \beta - y)^\top(X \beta - y)$

In [5]:
# Ridge Regression (RR)
def solveRR(y, X, lam):
    n, p = X.shape
    assert (len(y) == n)
    
    A = X.T.dot(X)
    # Adjust diagonal due to Ridge
    A[np.diag_indices_from(A)] += lam * n
    b = X.T.dot(y)
    
    # Hint:
    beta = np.linalg.solve(A, b)
    # Finds solution to the linear system Ax = b
    return (beta)

**Try it out:**

In [6]:
lam = 0.1

# Our solver
beta1 = solveRR(y, X, lam)

# Python solver
alpha = lam * X.shape[0]
model = lm.Ridge(alpha=alpha, fit_intercept=False, normalize=False)
beta2 = model.fit(X, y).coef_

# Check
compare(beta1, beta2)


Our solver:
[ 1.27929172  0.78935356  0.05064497 -0.55474398  0.65276533  0.32637554
  0.765293    0.63326617  0.97285396 -0.5294559 ]
Scikit-learn:
[ 1.27929172  0.78935356  0.05064497 -0.55474398  0.65276533  0.32637554
  0.765293    0.63326617  0.97285396 -0.5294559 ]

Difference between the two:
8.6474254503e-32
        


***
### Weighted Ridge Regression (WRR)

Given $X \in \mathbb{R}^{n \times p}$ and $y \in \mathbb{R}^n$, and weights $w \in \mathbb{R}^n_+$, solve
$$
\min_{\beta \in \mathbb{R}^p} \frac{1}{n} \sum_{i=1}^n w_i (y_i - \beta^\top x_i)^2 + \lambda \|\beta\|^2 \,.
$$

You can think of $w_i$ as importance or confidence you have in point $i$
$$$$

Since the weights $w_i$ are non-negative, we can pull $w_i$ inside the parenthesis:
$$\sum_{i=1}^n w_i (y_i - \beta^\top x_i)^2 = \sum_{i=1}^n  (\sqrt{w_i}y_i - \beta^\top \sqrt{w_i} x_i)^2$$

This is a regular Ridge Regression (RR)!

with <font color='green'> $y_i' = \sqrt{w_i}y_i$</font> and <font color='green'>$x_i' = \sqrt{w_i}x_i$ </font>
$$$$

#### In matrix form:

Introducing the diagonal matrix $W = \mathrm{diag}(w_1, ..., w_n)$, we can rewrite the objective
$$\sum_{i=1}^n w_i (y_i - \beta^\top x_i)^2 = (Y - X\beta)^\top W (Y - X\beta)$$

We now write $W = W^\frac{1}{2}W^\frac{1}{2} = (W^\frac{1}{2})^\top W^\frac{1}{2}$, where $W^\frac{1}{2} = \mathrm{diag}(\sqrt{w_1}, ..., \sqrt{w_n})$
$$$$

The objective becomes:
$$
\frac{1}{n} (W^\frac{1}{2}Y - W^\frac{1}{2}X\beta)^\top  (W^\frac{1}{2}Y - W^\frac{1}{2}X\beta) + \lambda \|\beta\|^2 = \frac{1}{n} \|Y' - X'\beta\|^2 + \lambda \|\beta\|^2 \,
$$
with <font color='green'>$Y' = W^\frac{1}{2} Y$</font> and <font color='green'>$X' = W^\frac{1}{2} X$</font>

In [7]:
# Weighted Ridge Regression (WRR)
def solveWRR(y, X, w, lam):
    n, p = X.shape
    assert (len(y) == len(w) == n)
    
    y1 = np.sqrt(w) * y
    X1 = (np.sqrt(w) * X.T).T
    # Hint:
    # Find y1 and X1 such that:
    beta = solveRR(y1, X1, lam)
    return (beta)

**Try it out:**

In [8]:
lam = 0.1
w = np.random.rand(len(y))

# Our solver
beta1 = solveWRR(y, X, w, lam)

# Python solver
alpha = lam * X.shape[0]
model = lm.Ridge(alpha=alpha, fit_intercept=False, normalize=False)
beta2 = model.fit(X, y, sample_weight=w).coef_

# Check
compare(beta1, beta2)


Our solver:
[ 1.21907902  0.69464471  0.05119819 -0.50155617  0.60130538  0.2718474
  0.6571524   0.60033163  0.862139   -0.50913719]
Scikit-learn:
[ 1.21907902  0.69464471  0.05119819 -0.50155617  0.60130538  0.2718474
  0.6571524   0.60033163  0.862139   -0.50913719]

Difference between the two:
4.93519548249e-32
        


***
### Logistic Ridge Regression (LRR)

Given $X \in \mathbb{R}^{n \times p}$ and $y \in \{-1,+1\}^n$, solve
$$
\min_{\beta \in \mathbb{R}^p} \frac{1}{n} \sum_{i=1}^n \log (1+e^{-y_i \beta^\top x_i}) + \lambda \|\beta\|^2 \,.
$$

Let $\sigma(x) = \frac{1}{1 + e^{-x}}$ be the sigmoid function.

Compute $\sigma'(x)$

$$\sigma'(x) = \frac{e^{-x}}{(1+e^{-x})^2} = \frac{1}{1+e^{-x}} - \frac{1}{(1+e^{-x})^2} = \sigma(x)(1-\sigma(x)) = \sigma(x)\sigma(-x)$$

**Note:** Under the logistic model, <font color='green'>$\sigma(y_i\beta^\top x_i) = \mathbb{P}[y=y_i \mid x_i, \beta]$</font>

- $\mathbb{P}[y_i = 1 \mid x_i, \beta] = \sigma(\beta^\top x_i)$ (definition of the model)
- $\mathbb{P}[y_i = 0 \mid x_i, \beta] = 1 - \sigma(\beta^\top x_i) = \sigma(-\beta^\top x_i)$

Rewriting $J$:
$$
J(\beta) = - \frac{1}{n} \sum_{i=1}^n {\log(\sigma(y_i\beta^\top x_i))} + \lambda \|\beta\|^2 \,.
$$

Compute its gradient $\nabla J$, and its Hessian $\nabla^2 J$
$$$$

#### Computing the gradient:
$$
\nabla J(\beta) = - \frac{1}{n} \sum_{i=1}^n {y_i \sigma(-y_i\beta^\top x_i) x_i} + 2\lambda \beta \,.
$$

In matrix form, with $P(\beta) = \mathrm{diag}\left[\sigma(-y_i\beta^\top x_i)\right]$:
<font color='green'>
$$
\nabla J(\beta) = - \frac{1}{n} X^T P(\beta)y + 2\lambda \beta \,.
$$
</font>

#### Hessian matrix:
$$
\nabla^2 J(\beta) = \nabla \left(\nabla J(\beta)\right)
$$

$$
\nabla^2 J(\beta) = - \frac{1}{n} \sum_{i=1}^n {\sigma(y_i\beta^\top x_i)\sigma(-y_i\beta^\top x_i) x_i x_i^\top} + 2\lambda I \,
$$

Define $w_i = \sigma(y_i\beta^\top x_i)\sigma(-y_i\beta^\top x_i)$ and $W = \mathrm{diag}(w_1, ..., w_n)$
$$$$
<font color='green'>
$$
\nabla^2 J(\beta) = - \frac{1}{n} X^\top W X + 2\lambda I \,
$$
</font>

***
**Note: Gradient descent:**
$$
\beta^{new} \leftarrow \beta^{old} - h \nabla J(\beta^{old})
$$
where $h$ is the step size (learning rate)
***

#### Solving for optimal $\beta$ using Newton-Raphson
$$
\beta^{new} \leftarrow \beta^{old} - \left(\nabla^2 J(\beta^{old})\right)^{-1} \nabla J(\beta^{old})
$$

This algorithm converges to the optimal $\beta$ (the one that minimizes $J$)
$$$$

$\approx$ gradient descent with adapted step size

Show that each step is equivalent to solving a weighted ridge regression (WRR). Hence we can compute $\beta$ without inverting the Hessian.


In one dimension:
$$
f(x) \approx f(x_0) + f'(x_0)(x-x_0) + \frac{1}{2}f"(x_0)(x-x_0)^2
$$

<font color='green'>Quadratic approximation to $J$</font>:

$$
J(\beta) \approx J_q(\beta) := J(\beta^{old}) + (\beta - \beta^{old})^\top \nabla J(\beta^{old}) + \frac{1}{2} (\beta - \beta^{old})^\top \nabla^2 J(\beta^{old}) (\beta - \beta^{old})
$$

**lemma**: $\min_\beta J_q(\beta) = \beta^{new} = \beta^{old} - \left(\nabla^2 J(\beta^{old})\right)^{-1} \nabla J(\beta^{old})$

*proof*: $$\nabla J_q(\beta) = \nabla J(\beta^{old}) + \nabla^2 J(\beta^{old}) (\beta - \beta^{old})$$
$$\nabla J_q(\beta) = 0 \iff \beta = \beta^{old} - \left(\nabla^2 J(\beta^{old})\right)^{-1} \nabla J(\beta^{old})$$

Next **show that $J_q$ is a WRR objective**

Let us write all the terms in $J_q$ that depend on $\beta$:
$$$$

- $\beta^\top \nabla J(\beta^{old}) = -\frac{1}{n} \beta^\top X^\top P(\beta^{old}) y + 2\lambda \beta^\top \beta^{old}$
- -$\beta^\top \nabla^2 J(\beta^{old}) \beta^{old} = -\frac{1}{n} \beta^\top X^\top W(\beta^{old}) X \beta^{old} - 2\lambda \beta^\top \beta^{old}$
- $\frac{1}{2}\beta^\top \nabla^2 J(\beta^{old}) \beta = \frac{1}{2n} \beta^\top X^\top W(\beta^{old}) X \beta + \lambda \beta^\top \beta$

Putting it all together:
$$2J_q(\beta) = \frac{1}{n} \beta^\top X^\top W X \beta - \frac{2}{n} \beta^\top X^\top W \left(X \beta^{old} + W^{-1}Py\right) + 2\lambda \|\beta\|^2 + \mathrm{Constant}$$

We now define $z = X \beta^{old} + W^{-1}Py$ and we can rewrite $J_q(\beta)$ as:
$$$$
<font color='green'>
$$
2J_q(\beta) = (z - X\beta)^\top W (z - X\beta) + 2\lambda \|\beta\|^2 + \mathrm{C}
$$
</font>

We recognize here a **weighted ridge regression!**

***
### Recap for IRLS:

- **Goal:** Minimize logistic loss
$$
J(\beta) = - \frac{1}{n} \sum_{i=1}^n {\log(\sigma(y_i\beta^\top x_i))} + \lambda \|\beta \|^2\,.
$$
- **Method:** Newton-Raphson

Start with $\beta^{old} = \beta_0$.

Then update until convergence:
$$
\beta^{new} \leftarrow \beta^{old} - \left(\nabla^2 J(\beta^{old})\right)^{-1} \nabla J(\beta^{old})
$$

Or **equivalently**
$$
\beta^{new} \leftarrow \min_\beta J_q(\beta)
$$
where 
$$
2J_q(\beta) = (z - X\beta)^\top W (z - X\beta) + 2\lambda \|\beta\|^2 + \mathrm{C}
$$
Each step of the Newton-Raphson, we solve a weighted Ridge regression!


Notations used:
- $P = \mathrm{diag}\left[\sigma(-y_i\beta^\top x_i)\right]$
- $W = \mathrm{diag}(w_1, ..., w_n)$ with $w_i = \sigma(y_i\beta^\top x_i)\sigma(-y_i\beta^\top x_i)$
- $z = X \beta^{old} + W^{-1}Py$
***

***
#### Recap for IRLS (code):

- Set:
    - $f_i = (\beta^{old})^\top x_i$
    - $W = \mathrm{diag}\left[\sigma(f_i)\sigma(-f_i)\right] = \mathrm{diag}\left[\sigma(f_1)\sigma(-f_1), ..., \sigma(f_n)\sigma(-f_n)\right]$
    - $z_i = f_i + \frac{y_i} {\sigma(y_i f_i)}$

    *(You can check that $\frac{-\sigma(y_i f_i)}{\sigma(f_i)\sigma(-f_i)} = \frac{1} {\sigma(y_i f_i)}$*)

- Call `solveWRR(z, W, X, 2`$\lambda$`)` at each update

- Stop when some criterion is reached, for example:
    - max number of iterations is reached
    - difference between 2 iterations is less than a threshold $\epsilon$: 
$$\|\beta^{new}-\beta^{old}\| < \epsilon$$
***

In [9]:
# Logistic Ridge Regression (LRR)
def solveLRR(y, X, lam):
    n, p = X.shape
    assert (len(y) == n)
    
    # Parameters
    max_iter = 500
    eps = 1e-3
    sigmoid = lambda a: 1/(1 + np.exp(-a))
    
    # Initialize
    beta = np.zeros(p)
            
    # Hint: Use IRLS
    for i in range(max_iter):
        beta_old = beta
        f = X.dot(beta_old)
        w = sigmoid(f) * sigmoid(-f)
        z = f + y / sigmoid(y*f)
        beta = solveWRR(z, X, w, 2*lam)
        # Break condition (achieved convergence)
        #if np.sum((beta-beta_old)**2) < eps:
        #    break
    return (beta)

**Try it out:**

In [10]:
y_bin = np.sign(y) # Binarize targets
lam = 0.1

# Our solver
beta1 = solveLRR(y_bin, X, lam)

# Python solver
alpha = 2 * lam * X.shape[0]
model = lm.LogisticRegression(C=1/alpha, fit_intercept=False)
beta2 = model.fit(X, y_bin).coef_

# Check
compare(beta1, beta2)


Our solver:
[ 0.49767153  0.31536881 -0.08695287 -0.16928258  0.27290435  0.15201944
  0.35505565  0.31972339  0.30170849 -0.39445252]
Scikit-learn:
[[ 0.4976808   0.31535489 -0.08695986 -0.16926035  0.27290697  0.15200484
   0.35506729  0.31972341  0.30172333 -0.39445142]]

Difference between the two:
1.40004809416e-09
        


***
### Mini Data Challenge

We will try to predict whether patients have breast cancer.

We use scikit-learn's [breast cancer dataset](https://scikit-learn.org/stable/datasets/index.html#breast-cancer-dataset)

30 features, 569 samples, 2 labels ('malignant' or 'benign')

In [11]:
# Load data and split into training / validation sets
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

X, y = load_breast_cancer(return_X_y=True)
# Scaling is important
X = sklearn.preprocessing.scale(X)
# X = (X - X.mean()) / X.std()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
X.shape

(569, 30)

In [12]:
# Fit our model and compute its parameters
lam = 0.01
beta = solveLRR(y_train, X_train, lam)

In [13]:
# Compute predicted probabilities and classes
probas_pred = 1/(1+np.exp(-beta.T.dot(X_test.T)))
y_pred = np.round(probas_pred)

In [14]:
from sklearn.metrics import accuracy_score, roc_auc_score

print("Our model's performance:")
print('Accuracy: {:.2%}'.format(accuracy_score(y_test, y_pred)))
print('AUC: {:.2%}'.format(roc_auc_score(y_test, probas_pred)))

Our model's performance:
Accuracy: 97.87%
AUC: 99.75%


In [15]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.96      0.99      0.97        67
          1       0.99      0.98      0.98       121

avg / total       0.98      0.98      0.98       188



In [16]:
import bqplot.pyplot as plt

plt.figure(title='Feature importances')
plt.bar(np.arange(len(beta)), beta)
plt.show()